In [7]:
import os
import pandas as pd
df = pd.read_csv(os.path.join('Data/Motor_Vehicle_Collisions_-_Crashes.csv'), dtype={'ZIP CODE': 'str'})
df[:50000].to_csv('Motor_Vehicle_Collisions_-_Crashes.csv')

In [5]:
df.head()

,CRASH DATE
0,04/14/2021
1,04/13/2021
2,04/15/2021
3,04/13/2021
4,04/12/2021


# Dependencies:

In [41]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
from sklearn.metrics import mean_absolute_error

from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.autogen.forecast_config import EvaluationPeriodParam
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.templates.autogen.forecast_config import ComputationParam

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

# Preparing the Dataframe:

In [42]:
cols = ['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'LATITUDE', 'LONGITUDE', 'NUMBER OF PERSONS INJURED', 
        'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 
        'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', ]

df = pd.read_csv('Data/Motor_Vehicle_Collisions_-_Crashes.csv', dtype={'ZIP CODE': 'str'},
                 usecols=cols)

df['CRASH DATE'] = pd.to_datetime((df['CRASH DATE'] + ' ' + df['CRASH TIME']), format="%m/%d/%Y %H:%M")

df_weather_temp = pd.read_csv('Data/Weather.csv')
df_weather_temp['datetime'] = pd.to_datetime(df_weather_temp['datetime'])

df_mta_temp = pd.read_csv('Data/MTA_Daily_Ridership_Data__Beginning_2020.csv', 
                        usecols=['Date', 'Subways: Total Estimated Ridership', 'Buses: Total Estimated Ridership', 
                                        'LIRR: Total Estimated Ridership', 'Metro-North: Total Estimated Ridership',
                                        'Bridges and Tunnels: Total Traffic'])
df_mta_temp.columns = ['Date', 'subway', 'bus', 'lirr', 'mnorth', 'bandt'] 
df_mta_temp['Date'] = pd.to_datetime(df_mta_temp['Date'])

In [46]:
df_p = df[['CRASH DATE', 'NUMBER OF PERSONS INJURED']]
df_p = df_p.resample(rule='D', on='CRASH DATE').sum().reset_index()
df_p.columns = ['ts', 'y']
df_p = df_p[(df_p['ts'] > '2020-04-01') & (df_p['ts'] < '2022-07-08')]

df_ = pd.DataFrame(index=pd.date_range(df_p.ts.max()+timedelta(days=1), periods=7, freq='D')).reset_index()
df_.columns = ['ts']
df_p = pd.concat([df_p, df_], join='outer')

df_weather_temp = df_weather_temp[['datetime', 'temp', 'solarradiation', 'windspeed']]
df_p = df_p.merge(df_weather_temp, how="left", left_on='ts', right_on='datetime')
df_p.drop(columns=['datetime'], inplace=True)

df_mta_temp = df_mta_temp[['Date', 'bandt']]
df_p = df_p.merge(df_mta_temp, how="left", left_on='ts', right_on='Date')
df_p.drop(columns=['Date'], inplace=True)

df_p['ylag7'] = df_p['y'].shift(7)
df_p['r30templag7'] = (df_p['temp'].rolling(30).mean()).shift(7)
df_p['templag7'] = df_p['temp'].shift(7)
df_p['templag8'] = df_p['temp'].shift(8)
df_p['solarradiationlag30'] = df_p['solarradiation'].shift(30)
df_p['windspeedlag15'] = df_p['windspeed'].shift(15)
df_p['bandtlag8'] = df_p['bandt'].shift(8)
df_p.dropna(subset=['ylag7', 'r30templag7', 'templag7', 'templag8', 'solarradiationlag30', 
                    'windspeedlag15', 'bandtlag8'], inplace=True)
df_p.drop(columns=['temp', 'solarradiation', 'windspeed', 'bandt'], inplace=True)

In [47]:
df_p.tail(8)

,ts,y,ylag7,r30templag7,templag7,templag8,solarradiationlag30,windspeedlag15,bandtlag8
826,2022-07-07,133.0,163.0,71.653333,78.2,74.7,331.0,14.1,977984.0
827,2022-07-08,NaN,153.0,72.276667,82.3,78.2,394.0,10.8,1011864.0
828,2022-07-09,NaN,140.0,72.730000,82.5,82.3,296.4,9.2,998259.0
829,2022-07-10,NaN,105.0,73.116667,80.7,82.5,405.6,7.6,823425.0
830,2022-07-11,NaN,126.0,73.313333,77.5,80.7,205.0,13.7,845877.0
831,2022-07-12,NaN,132.0,73.610000,77.0,77.5,210.3,13.2,772949.0
832,2022-07-13,NaN,131.0,73.866667,79.3,77.0,297.7,8.7,925869.0
833,2022-07-14,NaN,133.0,74.073333,76.2,79.3,330.5,10.8,917894.0


# Silverkite:

In [48]:
df_p['silverkite'] = 0
regressors=dict(regressor_cols=[['ylag7','r30templag7','templag7', 'templag8', 'solarradiationlag30', 
                                 'windspeedlag15', 'bandtlag8']])
model_components = ModelComponentsParam(regressors = regressors)

evaluation_period = EvaluationPeriodParam(cv_max_splits=1)

mae_train, mae_test = [],[]
main_predict_main = []
for i in range(0,13,1):
    metadata = MetadataParam(time_col="ts", value_col="y", freq="D", 
                             train_end_date=pd.to_datetime('2022-07-07')-datetime.timedelta(days=7*i))
 
    forecaster = Forecaster()
    result = forecaster.run_forecast_config(
    df = df_p,
    config = ForecastConfig(
        model_template = ModelTemplateEnum.SILVERKITE.name, forecast_horizon = 7, coverage = None, 
        metadata_param = metadata, 
        computation_param = ComputationParam(n_jobs=-1), 
        model_components_param=model_components,
        evaluation_period_param=evaluation_period))

    forecast = result.forecast
    res = forecast.df
    
    l = df_p.shape[0]
    df_p.iloc[l-(i+1)*7 : l-(i+1)*7+7, -1] = res.tail(7)['forecast']
    
    if i>0:
        l = res.shape[0]
        y_test = res.iloc[l-7:]['actual']
        y_predict = res.iloc[l-7:]['forecast']
        y_train = res.iloc[:l-7]['actual']
        y_train_predict = res.iloc[:l-7]['forecast']     
        mae_train.append(mean_absolute_error(y_train, y_train_predict))
        mae_test.append(mean_absolute_error(y_predict,y_test))

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits


In [49]:
df_p['silverkite_mean_train'] = np.round(np.mean(mae_train), 1)
df_p['silverkite_mean_test'] = np.round(np.mean(mae_test), 1)
df_p['silverkite_std_train'] = np.round(np.std(mae_train), 1)
df_p['silverkite_std_test'] = np.round(np.std(mae_test), 1)

# Prophet:

In [50]:
df_p['prophet'] = 0

regressors=dict(
    add_regressor_dict={"ylag7": {"prior_scale": 10.0, "mode": 'additive' },
                        "r30templag7": {"prior_scale": 10.0, "mode": 'additive'},
                        "templag7": {"prior_scale": 10.0, "mode": 'additive'},
                        "templag8": {"prior_scale": 10.0, "mode": 'additive'},
                        "solarradiationlag30": {"prior_scale": 10.0, "mode": 'additive'},
                        "windspeedlag15": {"prior_scale": 10.0, "mode": 'additive'},
                        "bandtlag8": {"prior_scale": 10.0, "mode": 'additive'},        
                        })

model_components = ModelComponentsParam(regressors = regressors)
evaluation_period = EvaluationPeriodParam(cv_max_splits=1)

mae_train, mae_test = [],[]
for i in range(0,13,1):
    metadata = MetadataParam(time_col="ts", value_col="y", freq="D", 
                             train_end_date=pd.to_datetime('2022-07-07')-datetime.timedelta(days=7*i))
 
    forecaster = Forecaster()
    result = forecaster.run_forecast_config(
    df = df_p,
    config = ForecastConfig(
        model_template = ModelTemplateEnum.PROPHET.name, forecast_horizon = 7, coverage = 0.95, 
        metadata_param = metadata, 
        computation_param = ComputationParam(n_jobs=-1), 
        model_components_param=model_components,
        evaluation_period_param=evaluation_period))

    forecast = result.forecast
    res = forecast.df
    
    l = df_p.shape[0]
    df_p.iloc[l-(i+1)*7 : l-(i+1)*7+7, -1] = res.tail(7)['forecast']
    
    if i>0:
        l = res.shape[0]
        y_test = res.iloc[l-7:]['actual']
        y_predict = res.iloc[l-7:]['forecast']

        y_train = res.iloc[:l-7]['actual']
        y_train_predict = res.iloc[:l-7]['forecast']     

        mae_train.append(mean_absolute_error(y_train, y_train_predict))
        mae_test.append(mean_absolute_error(y_predict,y_test))

Fitting 1 folds for each of 1 candidates, totalling 1 fits


16:57:15 - cmdstanpy - INFO - Chain [1] start processing
16:57:15 - cmdstanpy - INFO - Chain [1] done processing
16:57:18 - cmdstanpy - INFO - Chain [1] start processing
16:57:18 - cmdstanpy - INFO - Chain [1] done processing
16:57:20 - cmdstanpy - INFO - Chain [1] start processing
16:57:21 - cmdstanpy - INFO - Chain [1] done processing


Fitting 1 folds for each of 1 candidates, totalling 1 fits


16:57:23 - cmdstanpy - INFO - Chain [1] start processing
16:57:23 - cmdstanpy - INFO - Chain [1] done processing
16:57:25 - cmdstanpy - INFO - Chain [1] start processing
16:57:26 - cmdstanpy - INFO - Chain [1] done processing
16:57:28 - cmdstanpy - INFO - Chain [1] start processing
16:57:28 - cmdstanpy - INFO - Chain [1] done processing


Fitting 1 folds for each of 1 candidates, totalling 1 fits


...

16:58:31 - cmdstanpy - INFO - Chain [1] start processing
16:58:31 - cmdstanpy - INFO - Chain [1] done processing
16:58:34 - cmdstanpy - INFO - Chain [1] start processing
16:58:34 - cmdstanpy - INFO - Chain [1] done processing
16:58:36 - cmdstanpy - INFO - Chain [1] start processing
16:58:37 - cmdstanpy - INFO - Chain [1] done processing


Fitting 1 folds for each of 1 candidates, totalling 1 fits


16:58:39 - cmdstanpy - INFO - Chain [1] start processing
16:58:39 - cmdstanpy - INFO - Chain [1] done processing
16:58:42 - cmdstanpy - INFO - Chain [1] start processing
16:58:42 - cmdstanpy - INFO - Chain [1] done processing
16:58:44 - cmdstanpy - INFO - Chain [1] start processing
16:58:44 - cmdstanpy - INFO - Chain [1] done processing


Fitting 1 folds for each of 1 candidates, totalling 1 fits


16:58:46 - cmdstanpy - INFO - Chain [1] start processing
16:58:46 - cmdstanpy - INFO - Chain [1] done processing
16:58:49 - cmdstanpy - INFO - Chain [1] start processing
16:58:49 - cmdstanpy - INFO - Chain [1] done processing
16:58:51 - cmdstanpy - INFO - Chain [1] start processing
16:58:51 - cmdstanpy - INFO - Chain [1] done processing


In [51]:
df_p['prophet_mean_train'] = np.round(np.mean(mae_train), 1)
df_p['prophet_mean_test'] = np.round(np.mean(mae_test), 1)
df_p['prophet_std_train'] = np.round(np.std(mae_train), 1)
df_p['prophet_std_test'] = np.round(np.std(mae_test), 1)

# Linear Regression:

In [52]:
df_p['linreg'] = 0
y = df_p['y']
X = df_p[['ylag7','r30templag7','templag7', 'templag8', 'solarradiationlag30', 'windspeedlag15', 'bandtlag8']]
sc = StandardScaler()
scaler = sc.fit(X)
X = scaler.transform(X)

mae_train, mae_test = [],[]
for i in range(1,13,1):
    l = X.shape[0]
    X_train = X[ : l-(i+1)*7]
    X_test = X[l-(i+1)*7 : l-(i+1)*7+7]
    y_train = y[ : l-(i+1)*7]
    y_test = y[l-(i+1)*7 : l-(i+1)*7+7]
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    prediction_ = linreg.predict(X_test)
    df_p.iloc[l-(i+1)*7 : l-(i+1)*7+7, -1] = prediction_
    mae_train.append(mean_absolute_error(linreg.predict(X_train),y_train))
    mae_test.append(mean_absolute_error(prediction_,y_test))
l = X.shape[0]
X_train = X[ : l-7]
X_test = X[l-7 : l]
y_train = y[ : l-7]
y_test = y[l-7 : l]
linreg = LinearRegression()
linreg.fit(X_train, y_train)
df_p.iloc[l-7 : l, -1] = linreg.predict(X_test)

df_p['linreg_mean_train'] = np.round(np.mean(mae_train), 1)
df_p['linreg_mean_test'] = np.round(np.mean(mae_test), 1)
df_p['linreg_std_train'] = np.round(np.std(mae_train), 1)
df_p['linreg_std_test'] = np.round(np.std(mae_test), 1)

# Logistic Regression:

In [54]:
df_p['logreg'] = 0

y = df_p['y']
X = df_p[['ylag7','r30templag7','templag7', 'templag8', 'solarradiationlag30', 'windspeedlag15', 'bandtlag8']]
mae_train, mae_test = [],[]
for i in range(1,13,1):
    l = X.shape[0]
    X_train = X[ : l-(i+1)*7]
    X_test = X[l-(i+1)*7 : l-(i+1)*7+7]
    y_train = y[ : l-(i+1)*7]
    y_test = y[l-(i+1)*7 : l-(i+1)*7+7]
    logreg = LogisticRegression(C=0.1, max_iter=2000)
    logreg.fit(X_train, y_train)
    prediction_ = logreg.predict(X_test)
    df_p.iloc[l-(i+1)*7 : l-(i+1)*7+7, -1] = prediction_
    mae_train.append(mean_absolute_error(logreg.predict(X_train),y_train))
    mae_test.append(mean_absolute_error(prediction_,y_test))

l = X.shape[0]
X_train = X[ : l-7]
X_test = X[l-7 : l]
y_train = y[ : l-7]
y_test = y[l-7 : l]
logreg = LogisticRegression(C=0.1, max_iter=2000)
logreg.fit(X_train, y_train)
df_p.iloc[l-7 : l, -1] = logreg.predict(X_test)

df_p['logreg_mean_train'] = np.round(np.mean(mae_train), 1)
df_p['logreg_mean_test'] = np.round(np.mean(mae_test), 1)
df_p['logreg_std_train'] = np.round(np.std(mae_train), 1)
df_p['logreg_std_test'] = np.round(np.std(mae_test), 1)

# XGBoost:

In [56]:
df_p['xgb'] = 0
features = ['ylag7','r30templag7','templag7', 'templag8', 'solarradiationlag30', 'windspeedlag15', 'bandtlag8']
mae_train, mae_test = [],[]
for i in range(1,13,1):
    l = df_p.shape[0]
    df_train = df_p.iloc[ : l-(i+1)*7]
    df_test = df_p.iloc[l-(i+1)*7 : l-(i+1)*7+7]
    X_train, y_train = df_train[features], df_train['y']
    X_test, y_test = df_test[features], df_test['y']  
    xgb_model = xgb.XGBRegressor(objective="count:poisson", random_state=42, colsample_bytree=0.3, 
                                 learning_rate=0.140, max_depth=2, n_estimators=102, subsample=0.264, gamma=3.8)
    xgb_model.fit(X_train, y_train)
    prediction_ = xgb_model.predict(X_test)
    df_p.iloc[l-(i+1)*7 : l-(i+1)*7+7, -1] = prediction_
    mae_train.append(mean_absolute_error(xgb_model.predict(X_train),y_train))
    mae_test.append(mean_absolute_error(prediction_,y_test))

l = df_p.shape[0]
df_train = df_p.iloc[ : l-7]
df_test = df_p.iloc[l-7 : l]
X_train, y_train = df_train[features], df_train['y']
X_test, y_test = df_test[features], df_test['y']  
xgb_model = xgb.XGBRegressor(objective="count:poisson", random_state=42, colsample_bytree=0.3, learning_rate=0.140,
                            max_depth=2, n_estimators=102, subsample=0.264, gamma=3.8)
xgb_model.fit(X_train, y_train)
df_p.iloc[l-7 : l, -1] = xgb_model.predict(X_test)

df_p['xgb_mean_train'] = np.round(np.mean(mae_train), 1)
df_p['xgb_mean_test'] = np.round(np.mean(mae_test), 1)
df_p['xgb_std_train'] = np.round(np.std(mae_train), 1)
df_p['xgb_std_test'] = np.round(np.std(mae_test), 1)

In [66]:
df_p.tail(8)

,ts,y,ylag7,r30templag7,templag7,templag8,solarradiationlag30,windspeedlag15,bandtlag8,silverkite,...,logreg,logreg_mean_train,logreg_mean_test,logreg_std_train,logreg_std_test,xgb,xgb_mean_train,xgb_mean_test,xgb_std_train,xgb_std_test
826,2022-07-07,133.0,163.0,71.653333,78.2,74.7,331.0,14.1,977984.0,157.062842,...,144.0,25.3,18.4,1.0,8.1,157.810608,17.2,15.8,0.1,8.5
827,2022-07-08,NaN,153.0,72.276667,82.3,78.2,394.0,10.8,1011864.0,168.769617,...,153.0,25.3,18.4,1.0,8.1,156.497574,17.2,15.8,0.1,8.5
828,2022-07-09,NaN,140.0,72.730000,82.5,82.3,296.4,9.2,998259.0,156.793504,...,144.0,25.3,18.4,1.0,8.1,157.827316,17.2,15.8,0.1,8.5
829,2022-07-10,NaN,105.0,73.116667,80.7,82.5,405.6,7.6,823425.0,149.079019,...,142.0,25.3,18.4,1.0,8.1,143.273117,17.2,15.8,0.1,8.5
830,2022-07-11,NaN,126.0,73.313333,77.5,80.7,205.0,13.7,845877.0,142.347661,...,149.0,25.3,18.4,1.0,8.1,146.623611,17.2,15.8,0.1,8.5
831,2022-07-12,NaN,132.0,73.610000,77.0,77.5,210.3,13.2,772949.0,144.543445,...,144.0,25.3,18.4,1.0,8.1,142.415222,17.2,15.8,0.1,8.5
832,2022-07-13,NaN,131.0,73.866667,79.3,77.0,297.7,8.7,925869.0,153.150350,...,144.0,25.3,18.4,1.0,8.1,161.459641,17.2,15.8,0.1,8.5
833,2022-07-14,NaN,133.0,74.073333,76.2,79.3,330.5,10.8,917894.0,146.696730,...,152.0,25.3,18.4,1.0,8.1,155.362656,17.2,15.8,0.1,8.5


In [58]:
df_final_models = df_p.tail(7*13)
df_final_models.to_csv('Data/art.csv')